## Building a simple classifier for predicting room occupancy & publish it in Kusto

Open dataset from UCI Repository: __[Occupancy Detection](https://archive.ics.uci.edu/ml/datasets/Occupancy+Detection+)__

Experimental data used for binary classification (room occupancy) from Temperature,Humidity,Light and CO2.
Ground-truth occupancy was obtained from time stamped pictures that were taken every minute


In [ ]:
import pandas as pd
import datetime
import pickle
import binascii

In [ ]:
!pip install Kqlmagic -q --no-cache-dir --upgrade

In [ ]:
reload_ext Kqlmagic

### Retrieving the table for classification from Kusto

In [ ]:
%kql kusto://code;cluster='Help';database='Samples'

In [ ]:
# NOTE: to make hash() consistent set env. variable PYTHONHASHSEED=0
%env PYTHONHASHSEED=0

q = '''
OccupancyDetection
'''

fn = "df" + str(hash(q)) + ".pkl"
print("Cache file name: ", fn)

In [ ]:
fn = "df" + str(hash(q)) + ".pkl"
try:
    df = pd.read_pickle(fn)
    print("Load df from " + fn)
except:
    print("Execute query...")
    try:
        %kql res << -query q
        df = res.to_dataframe()
        print("Save df to " + fn)
        df.to_pickle(fn)
        print("\n", df.shape, "\n", df.columns)
    except Exception as ex:
        print(ex)

In [ ]:
print(df.shape, "\n")
print(df[-4:])

In [ ]:
df.groupby(['Test', 'Occupancy']).size()

## Train Model

In [ ]:
train_x = df[df['Test'] == False][['Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio']]
train_y = df[df['Test'] == False]['Occupancy']
test_x = df[df['Test'] == True][['Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio']]
test_y = df[df['Test'] == True]['Occupancy']

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

In [ ]:
from sklearn import tree
from sklearn import neighbors
from sklearn import naive_bayes
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

#four classifier types
clf1 = tree.DecisionTreeClassifier()
clf2 = LogisticRegression()
clf3 = neighbors.KNeighborsClassifier()
clf4 = naive_bayes.GaussianNB()

In [ ]:
clf1 = clf1.fit(train_x, train_y)
clf2 = clf2.fit(train_x, train_y)
clf3 = clf3.fit(train_x, train_y)
clf4 = clf4.fit(train_x, train_y)

#### Accuracy on Training set

In [ ]:
for clf, label in zip([clf1, clf2, clf3, clf4], ['Decision Tree', 'Logistic Regression', 'K Nearest Neighbour', 'Naive Bayes']):
            scores = cross_val_score(clf, train_x, train_y, cv=5, scoring='accuracy')
            print("Accuracy: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))

#### Accuracy on Testing set

In [ ]:
for clf, label in zip([clf1, clf2, clf3, clf4], ['Decision Tree', 'Logistic Regression', 'K Nearest Neighbour', 'Naive Bayes']):
            scores = cross_val_score(clf, test_x, test_y, cv=5, scoring='accuracy')
            print("Accuracy: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))

## Export the model to Kusto

In [ ]:
models_tbl = 'ML_Models_tmp'
model_name = 'Occupancy'

#### Create a dataframe containing model name, timestamp & the serialized model

In [ ]:
bmodel = pickle.dumps(clf2)
smodel = binascii.hexlify(bmodel)

now = datetime.datetime.now()
dfm = pd.DataFrame({'name':[model_name], 'timestamp':[now], 'model':[smodel]})
dfm

#### Store it in table of models

In [ ]:
set_query = '''
.set-or-append {0} <|
let tbl = dfm;
tbl
'''.format(models_tbl)
print(set_query)

In [ ]:
%kql -query set_query

## Test Model

#### Extract the last version of the named model from the table of models

In [ ]:
get_query = '''
let tbl_name = models_tbl;
let m_name = model_name;
table(tbl_name)
| where name == m_name
| top 1 by timestamp desc
'''
print(get_query)

In [ ]:
%kql res << -query get_query
model_df = res.to_dataframe()
qmodel = model_df.loc[0, 'model']

#### Create the trained model object and test it

In [ ]:
import pickle
import binascii

bmodel = binascii.unhexlify(qmodel)
clfp = pickle.loads(bmodel)
print(clfp)

In [ ]:
pscore = cross_val_score(clfp, test_x, test_y, cv=5, scoring='accuracy')
print("Accuracy: %0.4f (+/- %0.4f)" % (pscore.mean(), pscore.std()))

In [ ]:
clfp.predict(test_x[:5])